# Payments Lifecycle

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/payments_lifecycle/payments_lifecycle.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/payments_lifecycle/payments_lifecycle.ipynb)

## Business Scenario

Payments and customers evolve across Bronze, Silver, and Gold. You need a full lifecycle workflow that handles SCD2 and fact enrichment.

## Value Proposition

- End-to-end Bronze -> Silver -> Gold workflow
- SCD2 support for customer history
- Consistent fact table enrichment

---

## Goals

1. Load customer snapshots
2. Process payments
3. Build Gold facts


In [1]:
from pathlib import Path
from lakelogic import DataProcessor

def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        if base.name == name and base.exists():
            return base
        candidate = base / name
        if candidate.exists():
            return candidate
        candidate = base / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
        candidate = base / "lakelogic" / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
    return cwd / name

EXAMPLE_DIR = resolve_example_dir("payments_lifecycle")

for name in [
    "bronze_customers.parquet",
    "silver_customers.parquet",
    "gold_customers_scd2.parquet",
    "bronze_payments.parquet",
    "silver_payments.parquet",
    "gold_payments.parquet",
]:
    target = EXAMPLE_DIR / "data" / name
    if target.exists():
        target.unlink()

def contract_path(name: str) -> Path:
    return EXAMPLE_DIR / name

def preview_frame(df, title=None, limit=5):
    if title:
        print(title)
    if df is None:
        print("<empty>")
        return
    if hasattr(df, "head"):
        try:
            print(df.head(limit))
            return
        except Exception:
            pass
    if hasattr(df, "limit"):
        try:
            print(df.limit(limit))
            return
        except Exception:
            pass
    print(df)


## 2. Initialize Customers (Initial Load)
We process the first version of our customer data.

In [2]:
# 1. Bronze
cust_bronze = DataProcessor(contract=contract_path("01_bronze_customers.yaml"))
res = cust_bronze.run_source(str(EXAMPLE_DIR / "data" / "raw_customers_v1.csv"))
cust_bronze.materialize(res.good)

# 2. Silver
cust_silver = DataProcessor(contract=contract_path("02_silver_customers.yaml"))
res = cust_silver.run_source(str(EXAMPLE_DIR / "data" / "bronze_customers.parquet"))
cust_silver.materialize(res.good)

# 3. Gold (Creates SCD2 table)
cust_gold = DataProcessor(contract=contract_path("03_gold_customers.yaml"))
res = cust_gold.run_source(str(EXAMPLE_DIR / "data" / "silver_customers.parquet"))
cust_gold.materialize(res.good)

preview_frame(getattr(res, "good", None), title="Initial Customer Dimension (Gold):")


2026-02-15 12:20:58.119 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\raw_customers_v1.csv via polars


2026-02-15 12:20:58.450 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Customers Landing (Bronze)]


2026-02-15 12:20:58.468 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 3, Total (post-transform): 3, Good: 3, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%


2026-02-15 12:21:03.619 | INFO     | lakelogic.core.materialization:materialize_dataframe:987 - Materialized 3 rows to D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\bronze_customers.parquet


2026-02-15 12:21:03.632 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\bronze_customers.parquet via polars


...

2026-02-15 12:21:03.810 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\silver_customers.parquet via polars


2026-02-15 12:21:03.873 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Customer Dimension (Gold)]


2026-02-15 12:21:03.889 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 3, Total (post-transform): 3, Good: 3, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%


2026-02-15 12:21:04.069 | INFO     | lakelogic.core.materialization:materialize_dataframe:1099 - Materialized 3 rows to D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\gold_customers_scd2.parquet


Initial Customer Dimension (Gold):
shape: (3, 5)
┌─────────────┬─────────┬─────────────────────┬───────┬──────────────┐
│ customer_id ┆ name    ┆ email               ┆ plan  ┆ updated_at   │
│ ---         ┆ ---     ┆ ---                 ┆ ---   ┆ ---          │
│ str         ┆ str     ┆ str                 ┆ str   ┆ datetime[μs] │
╞═════════════╪═════════╪═════════════════════╪═══════╪══════════════╡
│ cust_3      ┆ Charlie ┆ charlie@example.com ┆ free  ┆ null         │
│ cust_2      ┆ Bob     ┆ bob@example.com     ┆ basic ┆ null         │
│ cust_1      ┆ Alice   ┆ alice@example.com   ┆ pro   ┆ null         │
└─────────────┴─────────┴─────────────────────┴───────┴──────────────┘


## 3. Process Payments (Joined with Customers)
Now we process payments, enriched with the initial customer metadata.

In [3]:
# 1. Bronze
pay_bronze = DataProcessor(contract=contract_path("01_bronze_payments.yaml"))
res = pay_bronze.run_source(str(EXAMPLE_DIR / "data" / "raw_payments.csv"))
pay_bronze.materialize(res.good)

# 2. Silver
pay_silver = DataProcessor(contract=contract_path("02_silver_payments.yaml"))
res = pay_silver.run_source(str(EXAMPLE_DIR / "data" / "bronze_payments.parquet"))
pay_silver.materialize(res.good)

# 3. Gold (Enriched via Join)
pay_gold = DataProcessor(contract=contract_path("03_gold_payments.yaml"))
res = pay_gold.run_source(str(EXAMPLE_DIR / "data" / "silver_payments.parquet"))
pay_gold.materialize(res.good)

preview_frame(getattr(res, "good", None), title="Enriched Payments (Gold):")


2026-02-15 12:21:04.110 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\raw_payments.csv via polars


2026-02-15 12:21:04.144 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Payments Landing (Bronze)]


2026-02-15 12:21:04.154 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 5, Total (post-transform): 5, Good: 4, Quarantined: 1, Pre-Transform Dropped: 0, Ratio: 20.00%


2026-02-15 12:21:04.201 | INFO     | lakelogic.core.materialization:materialize_dataframe:987 - Materialized 4 rows to D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\bronze_payments.parquet


2026-02-15 12:21:04.218 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\bronze_payments.parquet via polars


...

2026-02-15 12:21:04.340 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\silver_payments.parquet via polars


2026-02-15 12:21:04.371 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Transaction Fact (Gold)]


2026-02-15 12:21:04.471 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 4, Total (post-transform): 4, Good: 4, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%


2026-02-15 12:21:04.488 | INFO     | lakelogic.core.materialization:materialize_dataframe:987 - Materialized 4 rows to D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\gold_payments.parquet


Enriched Payments (Gold):
shape: (4, 9)
┌────────┬────────────┬─────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ raw_id ┆ customer_i ┆ gateway ┆ amount_str ┆ … ┆ amount_usd ┆ processed_ ┆ customer_ ┆ customer_ │
│ ---    ┆ d          ┆ ---     ┆ ---        ┆   ┆ ---        ┆ at         ┆ name      ┆ plan      │
│ str    ┆ ---        ┆ str     ┆ str        ┆   ┆ f64        ┆ ---        ┆ ---       ┆ ---       │
│        ┆ str        ┆         ┆            ┆   ┆            ┆ datetime[μ ┆ str       ┆ str       │
│        ┆            ┆         ┆            ┆   ┆            ┆ s]         ┆           ┆           │
╞════════╪════════════╪═════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ tx_004 ┆ cust_3     ┆ square  ┆ 12.99      ┆ … ┆ 12.99      ┆ null       ┆ Charlie   ┆ free      │
│ tx_003 ┆ cust_1     ┆ stripe  ┆ 200.00     ┆ … ┆ 200.0      ┆ null       ┆ Alice     ┆ pro       │
│ tx_002 ┆ cust_2     ┆ paypal  ┆ 45.00      ┆ … ┆ 

## 4. SCD Type 2: Customer Upgrade
Alice upgrades from 'pro' to 'enterprise'. We process the new version of customer data.
LakeLogic's `scd2` strategy will close the old record and open a new one.

In [4]:
# Process v2 customers by overriding the source path
res = cust_bronze.run_source(str(EXAMPLE_DIR / "data" / "raw_customers_v2.csv"))
cust_bronze.materialize(res.good)

res = cust_silver.run_source(str(EXAMPLE_DIR / "data" / "bronze_customers.parquet"))
cust_silver.materialize(res.good)

res = cust_gold.run_source(str(EXAMPLE_DIR / "data" / "silver_customers.parquet"))
cust_gold.materialize(res.good)

preview_frame(getattr(res, "good", None), title="Updated Customer Dimension (Gold SCD2):")


2026-02-15 12:21:04.515 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\raw_customers_v2.csv via polars


2026-02-15 12:21:04.538 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Customers Landing (Bronze)]


2026-02-15 12:21:04.553 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 2, Total (post-transform): 2, Good: 2, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%


2026-02-15 12:21:04.573 | INFO     | lakelogic.core.materialization:materialize_dataframe:987 - Materialized 5 rows to D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\bronze_customers.parquet


2026-02-15 12:21:04.580 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\bronze_customers.parquet via polars


...

2026-02-15 12:21:04.657 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\silver_customers.parquet via polars


2026-02-15 12:21:04.668 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Customer Dimension (Gold)]


2026-02-15 12:21:04.678 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 4, Total (post-transform): 4, Good: 4, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%


2026-02-15 12:21:06.286 | INFO     | lakelogic.core.materialization:materialize_dataframe:1099 - Materialized 7 rows to D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\gold_customers_scd2.parquet


Updated Customer Dimension (Gold SCD2):
shape: (4, 5)
┌─────────────┬─────────┬─────────────────────┬────────────┬──────────────┐
│ customer_id ┆ name    ┆ email               ┆ plan       ┆ updated_at   │
│ ---         ┆ ---     ┆ ---                 ┆ ---        ┆ ---          │
│ str         ┆ str     ┆ str                 ┆ str        ┆ datetime[μs] │
╞═════════════╪═════════╪═════════════════════╪════════════╪══════════════╡
│ cust_4      ┆ Diana   ┆ diana@example.com   ┆ pro        ┆ null         │
│ cust_1      ┆ Alice   ┆ alice@example.com   ┆ enterprise ┆ null         │
│ cust_3      ┆ Charlie ┆ charlie@example.com ┆ free       ┆ null         │
│ cust_2      ┆ Bob     ┆ bob@example.com     ┆ basic      ┆ null         │
└─────────────┴─────────┴─────────────────────┴────────────┴──────────────┘


## 5. Re-Run Payments (Picking up current dimension)
Since the business logic points to the 'current' dimension record, running the payments again will now show Alice as 'enterprise'.

In [5]:
res = pay_gold.run_source(str(EXAMPLE_DIR / "data" / "silver_payments.parquet"))
preview_frame(getattr(res, "good", None), title="Payments with UPDATED Customer Metadata:")


2026-02-15 12:21:06.322 | INFO     | lakelogic.core.processor:run_source:408 - Loading source: D:\Github\_SaaS\lakelogic\examples\06_advanced_workflows\payments_lifecycle\data\silver_payments.parquet via polars


2026-02-15 12:21:06.339 | INFO     | lakelogic.core.processor:run:278 - Starting LakeLogic run [Auto-Engine: polars, Contract: Transaction Fact (Gold)]


2026-02-15 12:21:06.378 | INFO     | lakelogic.core.processor:run:319 - Run complete. Source: 4, Total (post-transform): 4, Good: 4, Quarantined: 0, Pre-Transform Dropped: 0, Ratio: 0.00%


Payments with UPDATED Customer Metadata:
shape: (4, 9)
┌────────┬────────────┬─────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ raw_id ┆ customer_i ┆ gateway ┆ amount_str ┆ … ┆ amount_usd ┆ processed_ ┆ customer_ ┆ customer_ │
│ ---    ┆ d          ┆ ---     ┆ ---        ┆   ┆ ---        ┆ at         ┆ name      ┆ plan      │
│ str    ┆ ---        ┆ str     ┆ str        ┆   ┆ f64        ┆ ---        ┆ ---       ┆ ---       │
│        ┆ str        ┆         ┆            ┆   ┆            ┆ datetime[μ ┆ str       ┆ str       │
│        ┆            ┆         ┆            ┆   ┆            ┆ s]         ┆           ┆           │
╞════════╪════════════╪═════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ tx_004 ┆ cust_3     ┆ square  ┆ 12.99      ┆ … ┆ 12.99      ┆ null       ┆ Charlie   ┆ free      │
│ tx_003 ┆ cust_1     ┆ stripe  ┆ 200.00     ┆ … ┆ 200.0      ┆ null       ┆ Alice     ┆ enterpris │
│        ┆            ┆         ┆   